## Play-zone: Autogenerating image masks for segmentation training
This notebook autogenerates a mask for one image at a time - it can therefore be used for testing parameters, approaches to thresholding, visualising pixel intensity kernel density plots, and generally trying things out.  This is helpful for understanding the processes used to autogenerate masks in the notebook:
"11-1_AIMINN_autogenerate_segmentation_training_masks.ipynb"

## Imports

In [ ]:
import numpy as np
import os
from skimage import morphology, measure, draw, io, exposure
import matplotlib.pyplot as plt
from matplotlib.spines import Spine
import cv2
import seaborn as sns
import scipy

In [ ]:

INPUT_DIR = "" # folder containing all radiographs
FILENAME = "" # radiograph png file you wish to work with

os.chdir(INPUT_DIR)

In [ ]:
# load image
image = io.imread(FILENAME)

# display image and max pixel intensity
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.show()
print(f"Maximum pixel intensity in image: {np.amax(image)}")

## Choose a threshold value
#### ?automate this by taking max value of second derivative (this should correspond to local minimum)

In [ ]:
# density plot of pixel values to choose threshold value
p = sns.distplot(image.ravel(), hist = False, kde = True,
             kde_kws = {'linewidth': 3},)

plt.tick_params(colors='k', width=2)
plt.xlim(0, 255)
plt.ylim(bottom=0, top=0.02)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.yticks(np.arange(0, 0.025, step=0.005))
plt.title('Density Plot of pixel intensities')
plt.xlabel('Pixel intensity value')
plt.ylabel('Density')
plt.show()

In [ ]:
# Pull density values from seaborn distplot
pixel_values = p._axes.lines[0].get_xdata()
density = p._axes.lines[0].get_ydata()

In [ ]:
# identify salient troughs in density plot
troughs, _ = scipy.signal.find_peaks(-density, distance=20, height=(-0.1,-0.001))

MINIMUM_INTENSITY = 150
TROUGH_INDEX = -2

trough_intensities = pixel_values[troughs]
troughs_above_min = [trough_intensity for trough_intensity in trough_intensities 
                     if trough_intensity > MINIMUM_INTENSITY]
print(f"All troughs identified : {troughs}")
print (f"Number of troughs above minimum of {MINIMUM_INTENSITY}: {len(troughs_above_min)}")

# identify the single trough to be used for thresholding 

if len(troughs_above_min) > 1:
    THRESHOLD=troughs_above_min[TROUGH_INDEX]
    print(f"Autothreshold intensity chosen: {THRESHOLD}")
elif len(troughs_above_min) == 1:
    THRESHOLD=troughs_above_min
    print(f"Autothreshold intensity chosen: {THRESHOLD}")
elif len(troughs_above_min) == 0:
    THRESHOLD= trough_intensities[-1]
    print(f"CAUTION: NO TROUGHS IN RANGE ABOVE {MINIMUM_INTENSITY}")
    print(f"Autothreshold intensity chosen: {THRESHOLD}")   

## Thresholding, selection of largest area in image, dimension of bounding box

In [ ]:
# Return array of trues and falses
image_thresh = image >= THRESHOLD

In [ ]:
# skimage.measure.label labels all continuous non-zero areas with a numerical label.
# skimage.measure.regionprops generates a dictionary of properties about each labelled region
image_labelled = np.array(measure.label(image_thresh))
props = measure.regionprops(image_labelled)

# Create a list of the areas of each labelled region
areas = []
for index, prop in enumerate(props):
    area = (props[index].area)
    areas.append(area)
    
max_area = max(areas)

# Remove all except largest object for hips (for knees this needs changing to all except largest two objects)
image_minus_smallobjects = morphology.remove_small_objects(image_labelled, min_size=max_area)

# Fill in unnecessary holes generated from the process so far
image_implant_only = (morphology.remove_small_holes(image_minus_smallobjects, area_threshold=500)).astype(int)

# Display thresholded image
io.imshow(image_implant_only)

# Sanity to check to ensure only 1 area remains
props_new = measure.regionprops(image_implant_only)
print(f"Number of remaining areas: {len(props_new)}")

In [ ]:
# Calculate dimensions of bounding box around implant
bound_box_corners = props_new[0].bbox
min_row, min_col, max_row, max_col = bound_box_corners
print(f"Bounding box: Rows {min_row} to {max_row} | Columns {min_col} to {max_col}")

##  Bounding box used to crop radiograph

In [ ]:
# crop image based on bounding dimensions above
image_cropped = image[min_row:max_row, min_col:max_col]
# image_cropped = exposure.equalize_adapthist(image_cropped, clip_limit=0.03)

print("Cropped image:")
plt.imshow(image_cropped, cmap='gray')
plt.axis('off')
plt.show()

## Mask used to crop implant, then equalisation applied

In [ ]:
# remove background from image with threshold
image_thresholded = np.multiply(image_implant_only, image)

# crop image using bounding box dimensions determined a few cells above
image_thresholded_cropped = image_thresholded[min_row:max_row, min_col:max_col]

print("Without equalisation:")
plt.imshow(image_thresholded_cropped, cmap='gray')
plt.axis('off')
plt.show()

# apply equalisation to more evenly distribute the pixel intensity histogram
image_equalised = exposure.equalize_adapthist(image_thresholded_cropped, clip_limit=0.03)

print("With equalisation:")
plt.imshow(image_equalised, cmap='gray')
plt.axis('off')
plt.show()